# Self-Contained Source Separation

## Setup

First let's import libraries and create some convenience functions for our neural network operations with sane default values.

Unfortunately there isn't an easy way to hide code blocks in jupyter, so just skip over this :)

In [1]:
%matplotlib inline
import tensorflow as tf
import matplotlib.pyplot as plt
from functools import partial
import numpy as np
import librosa
import os

plt.rcParams['figure.figsize'] = [20, 8]
plt.rcParams['image.cmap'] = 'hot'

C:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def concat(x, y):
    return tf.concat([x, y], axis=3)


def conv(inputs, filters, kernel_size, stride):
    out = tf.layers.conv2d(
        inputs, filters=filters, kernel_size=kernel_size,
        kernel_initializer=tf.random_normal_initializer(stddev=0.02),
        strides=stride, padding='SAME')

    return out


def deconv(inputs, filters, kernel_size, stride):
    out = tf.layers.conv2d_transpose(
        inputs, filters=filters, kernel_size=kernel_size,
        kernel_initializer=tf.random_normal_initializer(stddev=0.02),
        strides=stride, padding='SAME')

    return out


def batch_norm(inputs, is_training, reuse):
    return tf.contrib.layers.batch_norm(
        inputs,
        decay=0.9,
        updates_collections=None,
        epsilon=1e-5,
        scale=True,
        is_training=is_training,
        reuse=reuse)


def dropout(inputs, rate):
    return tf.nn.dropout(inputs, keep_prob=1 - rate)


def relu(inputs):
    return tf.nn.relu(inputs)


def tanh(inputs):
    return tf.nn.tanh(inputs)


def lrelu(x, leak=0.2):
    with tf.variable_scope('lrelu'):
        f1 = 0.5 * (1 + leak)
        f2 = 0.5 * (1 - leak)
        return f1 * x + f2 * abs(x)


def l1_loss(x, y):
    return tf.reduce_mean(tf.abs(x - y))


def read_audio(path, sample_rate, n_channels):

    def read_audio_py(py_path):
        #if n_channels == 1:
            mono, _ = librosa.load(py_path, sr=sample_rate, mono=True)
            return np.expand_dims(mono, 1)
        #elif n_channels == 2:
            #stereo, _ = librosa.load(py_path, sr=sample_rate, mono=False)
            #return stereo.T
        #else:
            #raise ValueError('Invalid channels: %d' % n_channels)

    return tf.py_func(read_audio_py, [path], tf.float32, stateful=False)


def fake_stereo(audio):

    def fake_stereo(x):
        return tf.stack([x, x], 1)
    
    voice = audio[:, 0]
    mixed = voice * 2
    return fake_stereo(mixed), fake_stereo(voice)


def compute_spectrogram(audio, n_fft, fft_hop, n_channels):
    '''
    Parameters
    ----------
    audio : single to dual channel audio shaped (n_samples, n_channels)

    Returns
    -------
    Tensor of shape (n_frames, 1 + n_fft / 2, n_channels * 2), where the
        last dimension is (left_mag, right_mag, left_phase, right_phase)
    '''

    def stft(x):
        spec = librosa.stft(
            x, n_fft=n_fft, hop_length=fft_hop, window='hann')
        # TODO: normalize?
        #mag = np.abs(spec)
        #temp = mag - mag.min()
        #mag_norm = temp / temp.max()
        return np.abs(spec), np.angle(spec)

    #def stereo_func(py_audio):
    #    left_mag, left_phase = stft(py_audio[:, 0])
    #    right_mag, right_phase = stft(py_audio[:, 1])
    #    ret = np.array([left_mag, right_mag, left_phase, right_phase]).T
    #    return ret.astype(np.float32)

    def mono_func(py_audio):
        mag, phase = stft(py_audio[:, 0])
        ret = np.array([mag, phase]).T
        return ret.astype(np.float32)

    #if n_channels == 2:
    #    func = stereo_func
    #elif n_channels == 1:
    #    func = mono_func
    #else:
    #    raise ValueError('Invalid channels: %d' % n_channels)

    with tf.name_scope('read_spectrogram'):
        ret = tf.py_func(mono_func, [audio], tf.float32, stateful=False)
        ret.set_shape([None, 1 + n_fft / 2, 2])   # n_channels * 2])
    return ret


def extract_spectrogram_patches(
        spec, n_fft, n_channels, patch_window, patch_hop):
    '''
    Parameters
    ----------
    spec : Spectrogram of shape (n_frames, 1 + n_fft / 2, n_channels * 2)

    Returns
    -------
    Tensor of shape (n_patches, patch_window, 1 + n_fft / 2, n_channels * 2)
        containing patches from spec.
    '''
    with tf.name_scope('extract_spectrogram_patches'):
        spec4d = tf.expand_dims(spec, 0)

        patches = tf.extract_image_patches(
            spec4d, ksizes=[1, patch_window, 1 + n_fft / 2, 1],
            strides=[1, patch_hop, 1 + n_fft / 2, 1],
            rates=[1, 1, 1, 1],
            padding='VALID'
        )

        num_patches = tf.shape(patches)[1]

        return tf.reshape(patches, [num_patches, patch_window,
                                    int(1 + n_fft / 2), 2])   # int(n_channels * 2)]) #here was '1 + n_fft / 2, n_channels * 2', 
                                                                              #it was causing an error in 6th code block

def replace_spectrogram_patches(patches):
    '''
    Parameters
    ----------
    Tensor of shape (n_patches, patch_window, 1 + n_fft / 2, n_channels * 2)
    containing patches from spec.

    Returns
    -------
    spec : Spectrogram of shape (n_frames, 1 + n_fft / 2, n_channels * 2)
    '''
    # Need to account for patch overlap
    # Is it possible to account for no set num patches per whole spectrogram?
    pass

def invert_spectrogram(mag, phase, n_fft, fft_hop):

    # Could be a problem if the spec has been normalised
    spec = np.array([mag.T, phase.T]).T
    wave = librosa.istft(spec, win_length=n_fft, hop_length=fft_hop, window='hann')

    return wave

def compute_cochleagram(audio, SAMPLE_RATE, low_lim, hi_lim):
    coch = cgram.human_cochleagram(audio, SAMPLE_RATE, low_lim, hi_lim, strict=False)
    
        
def hwr_tf(x):
    return x * tf.cast(x > 0.0, tf.float32)


def compute_acapella_diff(mixed, noise):
    mixed_mag = mixed[:, :, 0:, :2]
    mixed_phase = mixed[:, :, 0:, 2:]
    noise_mag = noise[:, :, 0:, :2]
    voice_mag = hwr_tf(mixed_mag - noise_mag) # TODO: normalize?
    voice_phase = mixed_phase
    return mixed, noise, tf.concat((voice_mag, voice_phase), axis=3)


def partial_argv(func, *args, **kwargs):
    '''
    Parameters
    ----------
    func    : A function that takes scalar argument and returns scalar value
    *args   : Args to partially apply to func
    *kwargs : Keyword args to partially apply to func

    Returns
    -------
    func(*args) : A function that maps func over *args and returns a tuple

    Example
    -------
    func = partial_argv(abs)
    func(-1, 2, -3, 4)

    # returns (1, 2, 3, 4)
    '''
    return lambda *other_args: tuple(map(partial(func, *args, **kwargs), other_args))


def zip_tensor_slices(*args):
    '''
    Parameters
    ----------
    *args : list of _n_ _k_-dimensional tensors, where _k_ >= 2
        The first dimension has _m_ elements.

    Returns
    -------
    result : Dataset of _m_ examples, where each example has _n_
        records of _k_ - 1 dimensions.

    Example
    -------
    ds = (
        tf.data.Dataset.zip((
            tf.data.Dataset.from_tensors([[1,2], [3,4], [5, 6]]),
            tf.data.Dataset.from_tensors([[10, 20], [30, 40], [50, 60]])
        ))
        .flat_map(zip_tensor_slices)  # <--- *HERE*
    )
    el = ds.make_one_shot_iterator().get_next()
    print sess.run(el)
    print sess.run(el)

    # Output:
    # (array([1, 2], dtype=int32), array([10, 20], dtype=int32))
    # (array([3, 4], dtype=int32), array([30, 40], dtype=int32))
    '''
    return tf.data.Dataset.zip(tuple([
        tf.data.Dataset.from_tensor_slices(arg)
        for arg in args
    ]))

Let's define a few constants we'll use throughout.

In [3]:
SAMPLE_RATE = 44100
N_FFT = 1024
FFT_HOP = 256
N_CHANNELS = 1   #was N_CHANNELS = 2
N_PARALLEL_READERS = 4
PATCH_WINDOW = 256
PATCH_HOP = 128
BATCH_SIZE = 8
N_SHUFFLE = 20
#low_lim = 50
#hi_lim = 5000

# With default params, each spectrum represents 0.02s, and hop length is 0.005s (so ~200frames/s)
# Each patch is roughly 1.25s long - this is a lot shorter than those used in the music paper

## U-Net Model

Next, we create our U-Nets, one for noise and one for voice. Each `UNetModel` has a `UNetEncoder` and a `UNetDecoder`.

In [4]:
class UNetModel(object):

    def __init__(self, mixed, voice, mixed_phase, is_training):
        self.mixed = mixed
        self.voice = voice
        self.mixed_phase = mixed_phase
        self.is_training = is_training

        self.voice_mask_unet = UNet(mixed, is_training=is_training, reuse=False, name='voice-mask-unet')

        self.voice_mask = self.voice_mask_unet.output

        self.gen_voice = self.voice_mask * mixed

        self.cost = l1_loss(self.gen_voice, voice)

        self.optimizer = tf.train.AdamOptimizer(
            learning_rate=0.0002,
            beta1=0.5,
        )
        self.train_op = self.optimizer.minimize(self.cost)


class UNet(object):

    def __init__(self, input_tensor, is_training, reuse, name):
        with tf.variable_scope(name, reuse=reuse):

            self.encoder = UNetEncoder(input_tensor, is_training, reuse)
            self.decoder = UNetDecoder(self.encoder.output, self.encoder, is_training, reuse)

            self.output = tanh(self.decoder.output) / 2 + .5


class UNetEncoder(object):

    def __init__(self, input_tensor, is_training, reuse):
        net = input_tensor
        with tf.variable_scope('encoder'):
            with tf.variable_scope('layer-1'):
                net = conv(net, filters=16, kernel_size=5, stride=(2, 2))
                self.l1 = net

            with tf.variable_scope('layer-2'):
                net = lrelu(net)
                net = conv(net, filters=32, kernel_size=5, stride=(2, 2))
                net = batch_norm(net, is_training=is_training, reuse=reuse)
                self.l2 = net

            with tf.variable_scope('layer-3'):
                net = lrelu(net)
                net = conv(net, filters=64, kernel_size=5, stride=(2, 2))
                net = batch_norm(net, is_training=is_training, reuse=reuse)
                self.l3 = net

            with tf.variable_scope('layer-4'):
                net = lrelu(net)
                net = conv(net, filters=128, kernel_size=5, stride=(2, 2))
                net = batch_norm(net, is_training=is_training, reuse=reuse)
                self.l4 = net

            with tf.variable_scope('layer-5'):
                net = lrelu(net)
                net = conv(net, filters=256, kernel_size=5, stride=(2, 2))
                net = batch_norm(net, is_training=is_training, reuse=reuse)
                self.l5 = net

            with tf.variable_scope('layer-6'):
                net = lrelu(net)
                net = conv(net, filters=512, kernel_size=5, stride=(2, 2))

            self.output = net


class UNetDecoder(object):

    def __init__(self, input_tensor, encoder, is_training, reuse):
        net = input_tensor

        with tf.variable_scope('decoder'):
            with tf.variable_scope('layer-1'):
                net = relu(net)
                net = deconv(net, filters=256, kernel_size=5, stride=(2, 2))
                net = batch_norm(net, is_training=is_training, reuse=reuse)
                net = dropout(net, .5)

            with tf.variable_scope('layer-2'):
                net = relu(concat(net, encoder.l5))
                net = deconv(net, filters=128, kernel_size=5, stride=(2, 2))
                net = batch_norm(net, is_training=is_training, reuse=reuse)
                net = dropout(net, .5)

            with tf.variable_scope('layer-3'):
                net = relu(concat(net, encoder.l4))
                net = deconv(net, filters=64, kernel_size=5, stride=(2, 2))
                net = batch_norm(net, is_training=is_training, reuse=reuse)
                net = dropout(net, .5)

            with tf.variable_scope('layer-4'):
                net = relu(concat(net, encoder.l3))
                net = deconv(net, filters=32, kernel_size=5, stride=(2, 2))
                net = batch_norm(net, is_training=is_training, reuse=reuse)


            with tf.variable_scope('layer-5'):
                net = relu(concat(net, encoder.l2))
                net = deconv(net, filters=16, kernel_size=5, stride=(2, 2))
                net = batch_norm(net, is_training=is_training, reuse=reuse)

            with tf.variable_scope('layer-6'):
                net = relu(concat(net, encoder.l1))
                net = deconv(net, filters=1, kernel_size=5, stride=(2, 2))

            self.output = net


## Data Pipelines

Next we define our data reading functions. We make use of [the Dataset API](https://www.tensorflow.org/api_docs/python/tf/data/Dataset).

In [5]:
def training_dataset(
        data_folder,
        sample_rate,
        n_fft,
        fft_hop,
        n_channels,
        patch_window,
        patch_hop,
        #batch_size,
        #n_shuffle,
        n_parallel_readers
):
    """Still need to fix this to stop it producing a tuple"""
    return (
        tf.data.Dataset.list_files(data_folder + '/*.wav', shuffle=False)
        .map(partial(
            read_audio,
            sample_rate=sample_rate,
            n_channels=n_channels
        ), num_parallel_calls=n_parallel_readers)
        #.map(fake_stereo, num_parallel_calls=n_parallel_readers)
        .map(partial_argv(
            compute_spectrogram,
            n_fft=n_fft,
            fft_hop=fft_hop,
            n_channels=n_channels,
        ), num_parallel_calls=n_parallel_readers)
        .map(partial_argv(
            extract_spectrogram_patches,
            n_fft=n_fft,
            n_channels=n_channels,
            patch_window=patch_window,
            patch_hop=patch_hop,
        ))
        .flat_map(zip_tensor_slices)
        #.batch(batch_size)
        #.shuffle(n_shuffle)
        #.repeat()
    )

def zip_datasets(dataset_a, dataset_b, n_shuffle, batch_size):
    return tf.data.Dataset.zip((dataset_a, dataset_b)).batch(batch_size).shuffle(n_shuffle).repeat()

Next let's create the Tensorflow [Session](https://www.tensorflow.org/programmers_guide/graphs).

In [6]:
tf.reset_default_graph()
sess = tf.Session()

And the dataset:

In [7]:
mixed_folder = 'C:/Users/Toby/MSc_Project/Test_Audio/GANdatasetsMini/train_sup/Mixed'
clean_folder = 'C:/Users/Toby/MSc_Project/Test_Audio/GANdatasetsMini/train_sup/Voice'
mds = training_dataset(mixed_folder, SAMPLE_RATE, N_FFT, FFT_HOP, N_CHANNELS, PATCH_WINDOW, PATCH_HOP,  N_PARALLEL_READERS)
cds = training_dataset(clean_folder, SAMPLE_RATE, N_FFT, FFT_HOP, N_CHANNELS, PATCH_WINDOW, PATCH_HOP, N_PARALLEL_READERS)
ds = zip_datasets(mds, cds, BATCH_SIZE, N_SHUFFLE)
mixed, voice = ds.make_one_shot_iterator().get_next()

## Training

Now that we have both the model and the data pipeline we can actually train a model.

In [8]:
is_training = tf.placeholder(shape=(), dtype=bool)
mixed_mag = mixed[0][:, :, 1:, :2] # Yet more hacking to get around this tuple problem
mixed_phase = mixed[0][:, :, 1:, 2:]
voice_mag = voice[0][:, :, 1:, :2]

model = UNetModel(
    mixed_mag,
    voice_mag,
    mixed_phase,
    is_training
)

sess.run(tf.global_variables_initializer())

Now let's train a small number of iterations.

In [9]:
for i in range(100):
    _, cost = sess.run([model.train_op, model.cost], {model.is_training: True})
    print("            , {0}, {1}".format(i, cost)) #here we had code 'print "            ", i, cost', it was causing an error
                                                    #most likely related to python syntax changes in newer version

            , 0, 0.907840371131897
            , 1, 0.9170125722885132
            , 2, 0.8991769552230835
            , 3, 0.9209613800048828
            , 4, 0.906827449798584
            , 5, 0.9208844900131226
            , 6, 0.9021686315536499
            , 7, 0.9201065301895142
            , 8, 0.9121226072311401
            , 9, 0.8954215049743652
            , 10, 0.9003520011901855
            , 11, 0.9106419682502747
            , 12, 0.9132102131843567
            , 13, 0.905126690864563
            , 14, 0.9038329124450684
            , 15, 0.8804886937141418
            , 16, 0.8894723653793335
            , 17, 0.8998304605484009
            , 18, 0.8833758234977722
            , 19, 0.8846715688705444
            , 20, 0.8755289912223816
            , 21, 0.8850868940353394
            , 22, 0.8955572247505188
            , 23, 0.8848239779472351
            , 24, 0.8835453987121582
            , 25, 0.8751486539840698
            , 26, 0.8770691156387329
            , 

Now let's try to save the model.

In [10]:
saver = tf.train.Saver(tf.global_variables(), write_version=tf.train.SaverDef.V2)
checkpoint_folder = 'C:/Users/Toby/MSc_Project/MScFinalProjectCheckpoints/Checkpoint_test/Checkpoint_test'
saver.save(sess, checkpoint_folder, global_step=int(i))

'C:/Users/Toby/MSc_Project/MScFinalProjectCheckpoints/Checkpoint_test/Checkpoint_test-99'

We can inspect the contenst of the saved checkpoint:

In [11]:
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file

#i = 99
checkpoint_folder = 'C:/Users/Toby/MSc_Project/MScFinalProjectCheckpoints/Checkpoint_test/Checkpoint_test'
checkpoint = checkpoint_folder + '-' + str(i)
# List ALL tensors.
print_tensors_in_checkpoint_file(file_name=checkpoint, tensor_name='', all_tensors='')

beta1_power (DT_FLOAT) []
beta2_power (DT_FLOAT) []
voice-mask-unet/decoder/layer-1/BatchNorm/beta (DT_FLOAT) [256]
voice-mask-unet/decoder/layer-1/BatchNorm/beta/Adam (DT_FLOAT) [256]
voice-mask-unet/decoder/layer-1/BatchNorm/beta/Adam_1 (DT_FLOAT) [256]
voice-mask-unet/decoder/layer-1/BatchNorm/gamma (DT_FLOAT) [256]
voice-mask-unet/decoder/layer-1/BatchNorm/gamma/Adam (DT_FLOAT) [256]
voice-mask-unet/decoder/layer-1/BatchNorm/gamma/Adam_1 (DT_FLOAT) [256]
voice-mask-unet/decoder/layer-1/BatchNorm/moving_mean (DT_FLOAT) [256]
voice-mask-unet/decoder/layer-1/BatchNorm/moving_variance (DT_FLOAT) [256]
voice-mask-unet/decoder/layer-1/conv2d_transpose/bias (DT_FLOAT) [256]
voice-mask-unet/decoder/layer-1/conv2d_transpose/bias/Adam (DT_FLOAT) [256]
voice-mask-unet/decoder/layer-1/conv2d_transpose/bias/Adam_1 (DT_FLOAT) [256]
voice-mask-unet/decoder/layer-1/conv2d_transpose/kernel (DT_FLOAT) [5,5,256,512]
voice-mask-unet/decoder/layer-1/conv2d_transpose/kernel/Adam (DT_FLOAT) [5,5,256,512]

Now delete the variable 'model', close the session and reset the graph...

In [42]:
model = None
tf.reset_default_graph()

... start a new session and recreate our variables and model structure

In [43]:
sess = tf.Session()
mixed_folder = 'C:/Users/Toby/MSc_Project/Test_Audio/GANdatasetsMini/train_sup/Mixed'
clean_folder = 'C:/Users/Toby/MSc_Project/Test_Audio/GANdatasetsMini/train_sup/Voice'
mds = training_dataset(mixed_folder, SAMPLE_RATE, N_FFT, FFT_HOP, N_CHANNELS, PATCH_WINDOW, PATCH_HOP,  N_PARALLEL_READERS)
cds = training_dataset(clean_folder, SAMPLE_RATE, N_FFT, FFT_HOP, N_CHANNELS, PATCH_WINDOW, PATCH_HOP, N_PARALLEL_READERS)
ds = zip_datasets(mds, cds, BATCH_SIZE, N_SHUFFLE)
mixed, voice = ds.make_one_shot_iterator().get_next()

In [44]:
is_training = tf.placeholder(shape=(), dtype=bool)
mixed_mag = mixed[0][:, :, 1:, :2] # Yet more hacking to get around this tuple problem
mixed_phase = mixed[0][:, :, 1:, 2:]
voice_mag = voice[0][:, :, 1:, :2]

model = UNetModel(
    mixed_mag,
    voice_mag,
    mixed_phase,
    is_training
)

sess.run(tf.global_variables_initializer())

Put some data through before restoring ...

In [45]:
cost = sess.run([model.cost], {model.is_training: False})
print(cost)
#Show a small section of the weights:
var = [v for v in tf.trainable_variables() if v.name == "voice-mask-unet/encoder/layer-5/conv2d/kernel:0"][0]
var = sess.run(var)
print(var[0, 0, 0:3, 0:3])

[0.9039811]
[[-0.03537172  0.01470058 -0.00822074]
 [ 0.00713794 -0.00845241  0.018083  ]
 [ 0.02894733 -0.00187844 -0.01820544]]


Then restore the checkpoint and put some more data through

In [46]:
restorer = tf.train.Saver()
restorer.restore(sess, checkpoint)

INFO:tensorflow:Restoring parameters from C:/Users/Toby/MSc_Project/MScFinalProjectCheckpoints/Checkpoint_test/Checkpoint_test-99


In [47]:
cost = sess.run([model.cost], {model.is_training: False})
print(cost)
#Show a small section of the weights:
var = [v for v in tf.trainable_variables() if v.name == "voice-mask-unet/encoder/layer-5/conv2d/kernel:0"][0]
var = sess.run(var)
print(var[0, 0, 0:3, 0:3]) # Have confirmed that this produces the same numbers every time (when loading the same checkpoint)

[0.8456491]
[[-0.02912672 -0.01266418  0.01293571]
 [-0.00605743 -0.01404153 -0.00759478]
 [-0.01654004 -0.03171291 -0.0079945 ]]


Hopefully the cost comes out closer to that seen at the end of training!